In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from census import Census

# Census API Key
# from config import gkey
from config import ckey
url = Census(ckey, year=2018)

In [2]:
# Pulling census data
census_data = url.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "NAME": "Name", "state": "State"})

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income
0,27,Minnesota,5527358.0,37.9,68411.0,36245.0
1,28,Mississippi,2988762.0,37.2,43567.0,23434.0
2,29,Missouri,6090062.0,38.5,53560.0,29537.0
3,30,Montana,1041732.0,39.8,52559.0,29765.0
4,31,Nebraska,1904760.0,36.4,59116.0,31101.0


In [3]:
beer_df = pd.read_csv("./output_data/craft_beer_nv.csv")
beer_df.head()

,Name,id,Alcohol By Volume,International Bitterness Units,Standard Reference Method,Style,Category,Brewer,City,State,Latitude,Longitude,Region
0,Possession Porter,723,5.6,0,0,Porter,Irish Ale,Diamond Knot Brewery & Alehouse,Mukilteo,Washington,47.9485,-122.3050,West
1,Free Bike Amber,742,4.5,0,0,American-Style Amber/Red Ale,North American Ale,Skagit River Brewing,Mount Vernon,Washington,48.4192,-122.3350,West
2,Potrero ESB,3982,5.2,0,0,Special Bitter or Best Bitter,British Ale,21st Amendment Brewery Cafe,San Francisco,California,37.7825,-122.3930,West
3,Twisted Kilt Scotch Ale,4221,5.0,0,0,Scotch Ale,British Ale,Thirsty Dog Brewing,Akron,Ohio,41.0689,-81.5172,Midwest
4,HarvestÃ¶r Fresh Hop Ale,5243,7.2,0,0,American-Style Pale Ale,North American Ale,Lift Bridge Brewery,Stillwater,Minnesota,45.0565,-92.8222,Midwest


In [4]:
df = beer_df[['Brewer','City','State','Latitude','Longitude','Region']]
brewery_df = df.drop_duplicates(subset='Brewer', keep='first')
brewery_df

,Brewer,City,State,Latitude,Longitude,Region
0,Diamond Knot Brewery & Alehouse,Mukilteo,Washington,47.9485,-122.3050,West
1,Skagit River Brewing,Mount Vernon,Washington,48.4192,-122.3350,West
2,21st Amendment Brewery Cafe,San Francisco,California,37.7825,-122.3930,West
3,Thirsty Dog Brewing,Akron,Ohio,41.0689,-81.5172,Midwest
4,Lift Bridge Brewery,Stillwater,Minnesota,45.0565,-92.8222,Midwest
...,...,...,...,...,...,...
1815,Milwaukee Ale House,Milwaukee,Wisconsin,43.0334,-87.9093,Midwest
1839,Titletown Brewing,Green Bay,Wisconsin,44.5200,-88.0173,Midwest
1847,Metropolitan Brewing,Chicago,Illinois,41.9750,-87.6740,Midwest
1880,San Diego Brewing,San Diego,California,32.7922,-117.0990,West


In [6]:
# Configure gmaps with API key
from config import gkey
gmaps.configure(api_key=gkey)

In [24]:
#Data inputs for Mapping
brew_coord = brewery_df[["Latitude", "Longitude"]]
locations = beer_df[["Latitude", "Longitude"]]
abv = beer_df['Alcohol By Volume']

In [25]:
# US Brewery Locations
brew_layer = gmaps.symbol_layer(
    brew_coord, fill_color='rgba(33, 70, 250, 0.4)',
    stroke_color= 'rgba(53, 64, 122, 0.4)', scale=3)

In [54]:
#Map Settings
map_center = (38.4937,-98.3804)
fig = gmaps.figure(center=map_center, zoom_level=4)

abv_heat_layer = gmaps.heatmap_layer(locations, weights=abv, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


abv_heat_layer.gradient = [(255,255,255,0),
                           (20,168,99,1),
                          (231,225,31,0.83),
                          (255,46,0,0.83)]

In [55]:
#ABV Heatmap
fig.add_layer(abv_heat_layer)
fig.add_layer(brew_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Interactive US Beer Map
# Number of Breweries
# Population
# Top Styles
marker_locations = vacation_df[["Latitude", "Longitude"]]
city_list = vacation_df['City Name'].to_list()
country_list = vacation_df['Country'].to_list()

vacation_cities = [
    {'hotel':hotel_list[0], 'city':city_list[0], 'country':country_list[0]},
    {'hotel':hotel_list[1], 'city':city_list[1], 'country':country_list[1]},
    {'hotel':hotel_list[2], 'city':city_list[2], 'country':country_list[2]},
    {'hotel':hotel_list[3], 'city':city_list[3], 'country':country_list[3]},
    {'hotel':hotel_list[4], 'city':city_list[4], 'country':country_list[4]},
    {'hotel':hotel_list[5], 'city':city_list[5], 'country':country_list[5]}
]

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
city_info = [info_box_template.format(**city) for city in vacation_cities]

marker_layer = gmaps.marker_layer(marker_locations, info_box_content=city_info)

map_center = (35,0)
fig = gmaps.figure(center=map_center, zoom_level=1.5)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig